# Sentence to PClean query demo
This notebook demos the sentence-to-PClean pipeline. Given a sentence giving information about some doctor, it generates and runs queries about those people against the Medicare dataset used in Alex Lew's paper using a PClean model trained by Ian Limarta.


In [3]:
import gc
import json
import logging
from pathlib import Path
import sqlite3
import string
from typing import Any, Optional, TypeVar, Union

from IPython.display import display, Markdown
import lark
import nest_asyncio
import requests
import spacy
import torch
import transformers
from tqdm import tqdm
import vllm

from genparse import InferenceSetup, InferenceSetupVLLM

nest_asyncio.apply()

logger = logging.getLogger("notebook")

In [4]:
debug_sentences_path = Path().resolve() / "debug_sentences.jsonl"
debug_sentences = [json.loads(line) for line in debug_sentences_path.read_text(encoding="utf-8").splitlines() if line.strip()]

In [5]:
detailed_debug_sentences_path = Path().resolve() / "debug_sentences_detailed.jsonl"
detailed_debug_sentences = [json.loads(line) for line in detailed_debug_sentences_path.read_text(encoding="utf-8").splitlines() if line.strip()]

## Example inputs and outputs

Taken from the first ten sentences (rows 2-11) in the [GPT-4 tweets Marjorie Freedman generated][gpt4_tweets].

[gpt4_tweets]: https://docs.google.com/spreadsheets/d/1vq_HAdbFY079vYVOppkFLk_zojJ46MJKsy-TzUm7Q3M/edit

In [3]:
sentence_table_row_template = string.Template("| $sentence | <pre>$example_output</pre> |")
def show_sentences(sentence_data):
    lines = ["| Sentence | Example PClean preamble code |", "| --- | --- |"]
    for sentence_datum in sentence_data:
        lines.append(sentence_table_row_template.substitute(sentence=sentence_datum["sentence"], example_output=sentence_datum["example_output"].replace("\n", "<br>")))
    markdown = "\n".join(lines)
    display(Markdown(markdown))


show_sentences(debug_sentences)

| Sentence | Example PClean preamble code |
| --- | --- |
| Just had an insightful consultation with Dr. Kay Ryan in Baltimore. Feeling optimistic about my health! #HealthMatters #Baltimore | <pre>name_key = PClean.resolve_dot_expression(trace.model, :Obs, :name)<br>row_trace[name_key] = "Kay Ryan"<br>city_key = PClean.resolve_dot_expression(trace.model, :Obs, :city)<br>row_trace[city_key] = "Baltimore"</pre> |
| Dr. Kay Ryan's office at 321 Pine St. in Baltimore is so welcoming and efficient. Highly recommend! #CityCare | <pre>name_key = PClean.resolve_dot_expression(trace.model, :Obs, :name)<br>row_trace[name_key] = "Kay Ryan"<br>city_key = PClean.resolve_dot_expression(trace.model, :Obs, :city)<br>row_trace[city_key] = "Baltimore"<br>address_key = PClean.resolve_dot_expression(trace.model, :Obs, :address)<br>row_trace[address_key] = "321 Pine St."</pre> |
| Exploring Baltimore after an informative appointment with Dr. Kay Ryan. Loving this city's vibe! #BaltimoreAdventures #DoctorVisit | <pre>name_key = PClean.resolve_dot_expression(trace.model, :Obs, :name)<br>row_trace[name_key] = "Kay Ryan"<br>city_key = PClean.resolve_dot_expression(trace.model, :Obs, :city)<br>row_trace[city_key] = "Baltimore"</pre> |
| Dr. Kay Ryan, a neurologist, provided excellent care today. Grateful for her expertise! #Healthcare #BaltimoreDoctors | <pre>name_key = PClean.resolve_dot_expression(trace.model, :Obs, :name)<br>row_trace[name_key] = "Kay Ryan"<br>city_key = PClean.resolve_dot_expression(trace.model, :Obs, :city)<br>row_trace[city_key] = "Baltimore"<br>occupation_key = PClean.resolve_dot_expression(trace.model, :Obs, :occupation)<br>row_trace[occupation_key] = "neurologist"</pre> |
| Feeling reassured after my visit to Dr. Kay Ryan in Baltimore. She's truly exceptional! #FeelingGood #CityOfCharm | <pre>name_key = PClean.resolve_dot_expression(trace.model, :Obs, :name)<br>row_trace[name_key] = "Kay Ryan"<br>city_key = PClean.resolve_dot_expression(trace.model, :Obs, :city)<br>row_trace[city_key] = "Baltimore"</pre> |
| Had a great experience with Dr. Kay Ryan at 321 Pine St. in Baltimore. Her attention to detail is unmatched! #HealthCheck | <pre>name_key = PClean.resolve_dot_expression(trace.model, :Obs, :name)<br>row_trace[name_key] = "Kay Ryan"<br>city_key = PClean.resolve_dot_expression(trace.model, :Obs, :city)<br>row_trace[city_key] = "Baltimore"<br>address_key = PClean.resolve_dot_expression(trace.model, :Obs, :address)<br>row_trace[address_key] = "321 Pine St."</pre> |
| From diagnosis to treatment, Dr. Kay Ryan, a cardiologist, covered it all. Baltimore, you're lucky to have her! #MedicalCare | <pre>name_key = PClean.resolve_dot_expression(trace.model, :Obs, :name)<br>row_trace[name_key] = "Kay Ryan"<br>city_key = PClean.resolve_dot_expression(trace.model, :Obs, :city)<br>row_trace[city_key] = "Baltimore"<br>occupation_key = PClean.resolve_dot_expression(trace.model, :Obs, :occupation)<br>row_trace[occupation_key] = "cardiologist"</pre> |
| Just wrapped up my appointment with Dr. Kay Ryan in Baltimore. Her professionalism is top-notch. #HealthJourney | <pre>name_key = PClean.resolve_dot_expression(trace.model, :Obs, :name)<br>row_trace[name_key] = "Kay Ryan"<br>city_key = PClean.resolve_dot_expression(trace.model, :Obs, :city)<br>row_trace[city_key] = "Baltimore"</pre> |
| Dr. Kay Ryan's office in Baltimore is so efficient and welcoming. A great experience overall! #DoctorVisit #CityCare | <pre>name_key = PClean.resolve_dot_expression(trace.model, :Obs, :name)<br>row_trace[name_key] = "Kay Ryan"<br>city_key = PClean.resolve_dot_expression(trace.model, :Obs, :city)<br>row_trace[city_key] = "Baltimore"</pre> |
| Can't say enough good things about Dr. Kay Ryan, a pediatrician, in Baltimore. Truly an outstanding doctor! #Grateful | <pre>name_key = PClean.resolve_dot_expression(trace.model, :Obs, :name)<br>row_trace[name_key] = "Kay Ryan"<br>city_key = PClean.resolve_dot_expression(trace.model, :Obs, :city)<br>row_trace[city_key] = "Baltimore"<br>occupation_key = PClean.resolve_dot_expression(trace.model, :Obs, :occupation)<br>row_trace[occupation_key] = "pediatrician"</pre> |

## Helper code

### SpaCy code

In [4]:
# Supporting code for entity extraction.
_SPACY_MODEL_NAME = 'en_core_web_trf'
spacy_model = spacy.load(_SPACY_MODEL_NAME)
_PERSON_LABEL = 'PERSON'
_LOCATION_LABEL = 'LOC'
# spaCy labels list example thanks to Stack Overflow user 'russhoppa': https://stackoverflow.com/a/78252807
_SPACY_LABELS = spacy_model.get_pipe("ner").labels
assert _PERSON_LABEL in _SPACY_LABELS

T = TypeVar('T')

def _uniquify(items: list[T]) -> list[T]:
    """
    O(n^2) order-preserving uniquification.

    Fine for short lists like a single sentence's list of PERSON entities.
    """
    result = []
    for item in items:
        if item not in result:
            result.append(item)
    return result

def get_people(sentence: str) -> list[str]:
    return _uniquify(
        [ent.text for ent in spacy_model(sentence).ents if ent.label_ == _PERSON_LABEL]
    )

def get_locations(sentence: str) -> list[str]:
    return _uniquify(
        [ent.text for ent in spacy_model(sentence).ents if ent.label_ == _LOCATION_LABEL]
    )

def show_ents(sentence: str) -> None:
    print(sentence)
    print(spacy_model(sentence).ents)

for sentence_datum in debug_sentences:
    show_ents(sentence_datum["sentence"])
    print()

Just had an insightful consultation with Dr. Kay Ryan in Baltimore. Feeling optimistic about my health! #HealthMatters #Baltimore
(Kay Ryan, Baltimore, Baltimore)

Dr. Kay Ryan's office at 321 Pine St. in Baltimore is so welcoming and efficient. Highly recommend! #CityCare
(Kay Ryan, Baltimore, CityCare)

Exploring Baltimore after an informative appointment with Dr. Kay Ryan. Loving this city's vibe! #BaltimoreAdventures #DoctorVisit
(Baltimore, Kay Ryan)

Dr. Kay Ryan, a neurologist, provided excellent care today. Grateful for her expertise! #Healthcare #BaltimoreDoctors
(Kay Ryan, today)

Feeling reassured after my visit to Dr. Kay Ryan in Baltimore. She's truly exceptional! #FeelingGood #CityOfCharm
(Kay Ryan, Baltimore)

Had a great experience with Dr. Kay Ryan at 321 Pine St. in Baltimore. Her attention to detail is unmatched! #HealthCheck
(Kay Ryan, Baltimore)

From diagnosis to treatment, Dr. Kay Ryan, a cardiologist, covered it all. Baltimore, you're lucky to have her! #Medical

### Extracting the generated code from a response

In [6]:
def extract_code_from_response(text: str) -> str:
    fence = '```'
    start = text.index(fence) + len(fence)
    julia_bit = 'julia\n'
    json_bit = 'json\n'
    if text[start:].startswith(julia_bit):
        start += len(julia_bit)
    elif text[start:].startswith(json_bit):
        start += len(json_bit)
    end = text.rindex('```')
    result = text[start:end].strip()
    return result

In [7]:
# Test to confirm it works as intended
text = '''<|start_header_id|>assistant<|end_header_id|>

Here is the Julia code to query the PClean table of records about doctors based on the given input sentence:

```julia
name_key = PClean.resolve_dot_expression(trace.model, :Obs, :name)
row_trace[name_key] = "Kay Ryan"
city_key = PClean.resolve_dot_expression(trace.model, :Obs, :city)
row_trace[city_key] = "Baltimore"
```'''
extract_code_from_response(text)

'name_key = PClean.resolve_dot_expression(trace.model, :Obs, :name)\nrow_trace[name_key] = "Kay Ryan"\ncity_key = PClean.resolve_dot_expression(trace.model, :Obs, :city)\nrow_trace[city_key] = "Baltimore"'

### Sort posterior

In [8]:
def sort_posterior(posterior):
    return {inference: likelihood for inference, likelihood in sorted(posterior.items(), key=lambda t: (t[1], t[0]), reverse=True)}

### Aggregate likelihoods over extracted code

In [9]:
def get_aggregate_likelihoods(posterior: dict[str, float]) -> dict[str, float]:
    result = {}
    for inference, likelihood in posterior.items():
        code_only = extract_code_from_response(inference)
        result.setdefault(code_only, 0.0)
        result[code_only] += likelihood
    return sort_posterior(result)

### Get best inference

In [10]:
def get_best_inference(posterior: dict[str, float]) -> tuple[str, float]:
    return max(posterior.items(), key=lambda t: (t[1], t[0]))

### Running the server

In [11]:
_BATCH_SIZE = 1
# Ben LeBrun's WIP server running on GCP as of 2024-07-16
_DEFAULT_GENPARSE_INFERENCE_SERVER_URI = 'http://34.122.30.137:8888/infer'

def run_inference_server(
    prompt: str,
    *,
    proposal: str = 'character',
    batch_size: int = _BATCH_SIZE,
    max_tokens: int,
    n_particles: int,
    temperature: float = 1.0,
    grammar: str,
    genparse_url: str = _DEFAULT_GENPARSE_INFERENCE_SERVER_URI,
) -> dict[str, float]:
    """
    Run inference using a server.
    """
    params = {
        'prompt': prompt,
        'method': 'smc-standard',
        'n_particles': n_particles,
        'lark_grammar': grammar,
        'proposal_name': proposal,
        'proposal_args': {},
        'max_tokens': max_tokens,
        'temperature': temperature,
    }
    headers = {
        "Content-type": "application/json",
        "Accept": "application/json"
    }
    response = requests.post(genparse_url, json=params, headers=headers)

    return response

### PClean grammar

In [12]:
pclean_grammar = r"""
start: prefix julia_code suffix
prefix: "<|start_header_id|>assistant<|end_header_id|>" NL* FREE_TEXT? NL+ CODE_FENCE JULIA? NL
suffix: NL CODE_FENCE
julia_code: add_to_trace (NL+ add_to_trace)+

FREE_TEXT: /[a-zA-Z0-9.,-?!;: ]+/
CODE_FENCE: "```"
JULIA: "julia"
WS: " "
NL: "\n"
STRING: /"[a-zA-Z0-9. ]*"/

add_to_trace: get_key NL set_key_in_trace
# slightly overly restrictive but good enough 
get_key: trace_key_identifier WS* "=" WS* "PClean.resolve_dot_expression(trace.model, :Obs, " column_symbol ")"
# column_symbol: /:[a-z][a-z_]+/
column_symbol: ":" ("name" | "address" | "specialty" | "city")
set_key_in_trace: "row_trace[" trace_key_identifier "]" WS* "=" WS* STRING
trace_key_identifier: /[a-z][a-z_]+/
"""
pclean_parser = lark.Lark(pclean_grammar)

### PClean code template

In [13]:
pclean_template = string.Template(
    """
# Create a new row trace for the hypothetical row
row_trace = Dict{PClean.VertexID, Any}()
$preamble

# Add it to the trace
obs = trace.tables[:Obs].observations
row_id = gensym()
obs[row_id] = row_trace

samples = []
for _ in 1:$N
    # Perform a Partilce Gibbs MCMC move to change our current sample of the row
    PClean.run_smc(!(trace, :Obs, row_id, PClean.InferenceConfig(1, 10))
    # Accumulate the sample
    push!(samples, trace.tables[:Obs].rows[row_id][br_idx]
end

countmap(samples)
""".lstrip()
)

PCLEAN_DEFAULT_N_SAMPLES = 100

### PClean generation prompt

In [14]:
pclean_prompt = string.Template(
    """Write Julia code to query a PClean table of records about doctors based on the given input sentence.

In general, your output should look like pairs of lines:

```julia
blah_key = PClean.resolve_dot_expression(trace.model, :Obs, :blah_col)
row_trace[blah_key] = "Value of Blah as expressed in the sentence"
```

The dataset has just four columns to query:

- :name (the doctor's full name, first and last)
- :city (the city where the doctor practices)
- :address (the doctor's office address)
- :specialty (the doctor's specialty)

Please generate code to query all values specified in the sentence. Output the Julia code directly with no preamble or commentary. Write just two lines per column.

Input: Loved visiting Dr. Kay Ryan's neurology office at 256 Overflow St! No wait time at all. #Baltimore
Output: ```julia
name_key = PClean.resolve_dot_expression(trace.model, :Obs, :name)
row_trace[name_key] = "Kay Ryan"
address_key = PClean.resolve_dot_expression(trace.model, :Obs, :address)
row_trace[address_key] = "256 Overflow St"
specialty_key = PClean.resolve_dot_expression(trace.model, :Obs, :specialty)
row_trace[specialty_key] = "neurology"
city_key = PClean.resolve_dot_expression(trace.model, :Obs, :city)
row_trace[city_key] = "Baltimore"
```

Input: Dr. Pat Rogers's orthopedics office screwed us! Took our money and Kay gave us three minutes tops. #BaltimoreSucks
Output: ```julia
name_key = PClean.resolve_dot_expression(trace.model, :Obs, :name)
row_trace[name_key] = "Pat Rogers"
specialty_key = PClean.resolve_dot_expression(trace.model, :Obs, :specialty)
row_trace[specialty_key] = "orthopedics"
city_key = PClean.resolve_dot_expression(trace.model, :Obs, :city)
row_trace[city_key] = "Baltimore"
```

In other words, we query all values given in the sentence: The doctor's name, their office address, their specialty, and the city. If one of these is missing, we do not query on it.

Input: $sentence
Output:"""
)
# scrapped
# The doctor's name is $name. Their office address is $address. Their specialty is $specialty. The city is $city.

def format_people_prompt(sentence: str) -> str:
    return pclean_prompt.substitute(sentence=sentence)

### Running PClean

In [15]:
def run_pclean(code: str) -> Any:
    # TODO
    raise NotImplementedError()

### Displaying PClean query results

In [16]:
def display_query_results(query_results: Any) -> None:
    # TODO
    raise NotImplementedError()

### JSON generation prompt

In [17]:
json_prompt = string.Template(
    """Write a flat JSON object describing the doctor in the given input sentence.

In general, your output should look like:

```json
{
    "some_feature": "foo",
    "another_feature": "bar",
    ...
    "last_feature": "baz"
}
```

There are seven features we would like to extract from the sentence:

- "first_name" (the doctor's first name)
- "last_name" (the doctor's last name)
- "c2z3" (a code of the form XX-YYY where XX are letters and YYY are digits)
- "address" (the doctor's office address, "line 1" -- this is the street address)
- "address2" (additional address information giving a suite number, unit number, et cetera -- "line 2" of the address)
- "specialty" (the doctor's specialty)
- "legal_name" (the legal name of the doctor's business, practice, or employer)

Please generate a key-value pair for every such value specified in the sentence. Avoid nulls and empty strings. Omit any missing values. Do not write a script. Output the JSON itself directly with no preamble or commentary. The following examples illustrate how you should behave on the input sentence.

Input: John Smith's neurology office (Happy Brain Services LLC) at 512 Example Street Suite 3600 (CA-170) is terrible!
Output: ```json
{"first_name": "John", "last_name": "Smith", "specialty": "neurology", "legal_name": "Happy Brain Services LLC", "address": "512 Example Street", "c2z3": CA-170"}
```

Input: Loved visiting Dr. Kay Ryan's neurology office at 256 Overflow St! No wait time at all. #Baltimore #CA-170
Output: ```json
{"first_name": "Kay", "last_name": "Ryan", "specialty": "neurology", "address": "256 Overflow St", "c2z3": CA-170"}
```

Input: Dr. Pat Rogers's orthopedics office screwed us! Took our money and Kay gave us three minutes tops. #BaltimoreSucks
Output: ```json
{"first_name": "Pat", "last_name": "Rogers", "specialty": "orthopedics"}
```

The following is your input sentence. Produce the appropriate output.

Input: $sentence
Output:"""
)
# scrapped
# The doctor's name is $name. Their office address is $address. Their specialty is $specialty. The city is $city.

def format_json_prompt(sentence: str) -> str:
    return json_prompt.substitute(sentence=sentence)

print(json_prompt)

### JSON grammar

In [18]:
json_grammar = r"""
start: prefix json suffix
prefix: "<|start_header_id|>assistant<|end_header_id|>" NL* FREE_TEXT? NL+ CODE_FENCE JSON_TAG? NL
suffix: NL CODE_FENCE
json: "{" NL? WS* kv_pair ("," NL? WS* kv_pair)* NL? WS* "}"
kv_pair: COLUMN WS* ":" WS* STRING

FREE_TEXT: /[a-zA-Z0-9.,-?!;:' ]+/
CODE_FENCE: "```"
JSON_TAG: "json"
WS: " "
NL: "\n"
STRING: /"[a-zA-Z0-9-.,'& ]*"/
# dropped for now: "city"
COLUMN: "\"" ("first_name" | "last_name" | "address" | "address2" | "specialty" | "c2z3" | "legal_name") "\""
"""
json_parser = lark.Lark(json_grammar)

In [26]:
json_grammar = r"""
start: prefix json suffix
prefix: "<|start_header_id|>assistant<|end_header_id|>" NL* FREE_TEXT? NL+ CODE_FENCE JSON_TAG? NL
suffix: NL CODE_FENCE
json: "{" NL? WS* kv_pair ("," NL? WS* kv_pair)* NL? WS* "}"
#kv_pair: COLUMN WS* ":" WS* STRING
kv_pair: (NONEMPTY_COLUMN WS* ":" WS* NONEMPTY_STRING) | (EMPTYABLE_COLUMN WS* ":" WS* STRING)

FREE_TEXT: /[a-zA-Z0-9.,-?!;:' ]+/
CODE_FENCE: "```"
JSON_TAG: "json"
WS: " "
NL: "\n"
NONEMPTY_STRING: /"[a-zA-Z0-9-.,'& ]+"/
STRING: /"[a-zA-Z0-9-.,'& ]*"/
# dropped for now: "city"
NONEMPTY_COLUMN: "\"" ("first_name" | "last_name" | "address" | "specialty" | "c2z3" | "legal_name") "\""
EMPTYABLE_COLUMN: "\"address2\""
COLUMN: "\"" ("first_name" | "last_name" | "address" | "address2" | "specialty" | "c2z3" | "legal_name") "\""
"""
json_parser = lark.Lark(json_grammar)

# Demo

Example sentences:

1. Just had an awful consultation with Dr. Kay Ryan in Baltimore. My heart has never felt worse -- Kay knows nothing! Get another cardiologist! #HealthMatters #Baltimore
2. Dr. Kay Ryan's office at 384 Oak St. in Baltimore is so welcoming and efficient. Highly recommend! #CityCare

In [ ]:
MAX_NEW_TOKENS = 256
N_PARTICLES = 15
N_SAMPLES = 100
server_inference_params = {'max_tokens': MAX_NEW_TOKENS, 'n_particles': N_PARTICLES, 'grammar': pclean_grammar}

while True:
    sentence = input('Give me a sentence involving a (named) physician (ideally with specialty, address, or city name): ')
    if sentence == 'q':
        break

    print(f'Processing sentence `{sentence}`...')
    people = get_people(sentence)
    logger.debug(f'People: {people}')
    
    prompt = format_people_prompt(sentence=sentence)
    logger.debug(f'Prompt: {prompt}')

    try:
        response = run_inference_server(prompt, **server_inference_params)
        data = response.json()
    except json.DecodeError as e:
        print(f'ERROR: response status {response.status_code} - {response.text}. Parse error {e}.')
        continue
    aggregate_likelihoods = get_aggregate_likelihoods(data['posterior'])
    best_code, likelihood = get_best_inference(aggregate_likelihoods)
    full_code = pclean_template.substitute(preamble=best_code, N=N_SAMPLES)
    
    display(Markdown('Code:'))
    display(Markdown(f'```julia\n{full_code}\n```'))
    display(Markdown(f'Likelihood: {100 * likelihood:.2f}%'))
    
    try:
        results = run_pclean(full_code)
        display(Markdown('Query Results:'))
        display_query_results(results)
    except NotImplementedError:
        print("Sorry, we can't actually run queries yet. 😅")

Give me a sentence involving a (named) physician (ideally with specialty, address, or city name):  Just had an awful consultation with Dr. Kay Ryan in Baltimore. My heart has never felt worse -- Kay knows nothing! Get another cardiologist! #HealthMatters #Baltimore


Processing sentence `Just had an awful consultation with Dr. Kay Ryan in Baltimore. My heart has never felt worse -- Kay knows nothing! Get another cardiologist! #HealthMatters #Baltimore`...


Code:

```julia
# Create a new row trace for the hypothetical row
row_trace = Dict{PClean.VertexID, Any}()
name_key = PClean.resolve_dot_expression(trace.model, :Obs, :name)
row_trace[name_key] = "Kay Ryan"
city_key = PClean.resolve_dot_expression(trace.model, :Obs, :city)
row_trace[city_key] = "Baltimore"

# Add it to the trace
obs = trace.tables[:Obs].observations
row_id = gensym()
obs[row_id] = row_trace

samples = []
for _ in 1:100
    # Perform a Partilce Gibbs MCMC move to change our current sample of the row
    PClean.run_smc(!(trace, :Obs, row_id, PClean.InferenceConfig(1, 10))
    # Accumulate the sample
    push!(samples, trace.tables[:Obs].rows[row_id][br_idx]
end

countmap(samples)

```

Likelihood: 100.00%

...

Processing sentence `Dr. Kay Ryan's office at 384 Oak St. in Baltimore is so welcoming and efficient. Highly recommend! #CityCare`...


Code:

```julia
# Create a new row trace for the hypothetical row
row_trace = Dict{PClean.VertexID, Any}()
name_key = PClean.resolve_dot_expression(trace.model, :Obs, :name)
row_trace[name_key] = "Kay Ryan"
address_key = PClean.resolve_dot_expression(trace.model, :Obs, :address)
row_trace[address_key] = "384 Oak St"
city_key = PClean.resolve_dot_expression(trace.model, :Obs, :city)
row_trace[city_key] = "Baltimore"

# Add it to the trace
obs = trace.tables[:Obs].observations
row_id = gensym()
obs[row_id] = row_trace

samples = []
for _ in 1:100
    # Perform a Partilce Gibbs MCMC move to change our current sample of the row
    PClean.run_smc(!(trace, :Obs, row_id, PClean.InferenceConfig(1, 10))
    # Accumulate the sample
    push!(samples, trace.tables[:Obs].rows[row_id][br_idx]
end

countmap(samples)

```

Likelihood: 86.67%

Sorry, we can't actually run queries yet. 😅


## Debugging

### Running the LM unconstrained

In [20]:
model_id = 'meta-llama/Meta-Llama-3-8B-Instruct'

In [12]:
try:
    del model
    gc.collect()
except NameError:
    pass

In [13]:
# Taken from a magic number in the Genparse VLLM backend code.
_MAX_MODEL_LEN = 4096
# Why dtype=torch.float32? It's in the Genparse VLLM backend code.
model = vllm.LLM(model_id, dtype=torch.float32, max_model_len=_MAX_MODEL_LEN)

INFO 07-22 14:56:24 config.py:1214] Upcasting torch.bfloat16 to torch.float32.
INFO 07-22 14:56:24 llm_engine.py:161] Initializing an LLM engine (v0.5.0.post1) with config: model='meta-llama/Meta-Llama-3-8B-Instruct', speculative_config=None, tokenizer='meta-llama/Meta-Llama-3-8B-Instruct', skip_tokenizer_init=False, tokenizer_mode=auto, revision=None, rope_scaling=None, rope_theta=None, tokenizer_revision=None, trust_remote_code=False, dtype=torch.float32, max_seq_len=4096, download_dir=None, load_format=LoadFormat.AUTO, tensor_parallel_size=1, disable_custom_all_reduce=False, quantization=None, enforce_eager=False, kv_cache_dtype=auto, quantization_param_path=None, device_config=cuda, decoding_config=DecodingConfig(guided_decoding_backend='outlines'), seed=0, served_model_name=meta-llama/Meta-Llama-3-8B-Instruct)


Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


INFO 07-22 14:56:34 selector.py:136] Cannot use FlashAttention-2 backend for dtype other than torch.float16 or torch.bfloat16.
INFO 07-22 14:56:34 selector.py:51] Using XFormers backend.
INFO 07-22 14:56:52 selector.py:136] Cannot use FlashAttention-2 backend for dtype other than torch.float16 or torch.bfloat16.
INFO 07-22 14:56:52 selector.py:51] Using XFormers backend.
INFO 07-22 14:57:07 weight_utils.py:218] Using model weights format ['*.safetensors']
INFO 07-22 14:57:37 model_runner.py:160] Loading model weights took 29.9190 GB
INFO 07-22 14:57:42 gpu_executor.py:83] # GPU blocks: 736, # CPU blocks: 1024
INFO 07-22 14:57:44 model_runner.py:889] Capturing the model for CUDA graphs. This may lead to unexpected consequences if the model is not static. To run the model in eager mode, set 'enforce_eager=True' or use '--enforce-eager' in the CLI.
INFO 07-22 14:57:44 model_runner.py:893] CUDA graphs can take additional 1~3 GiB memory per GPU. If you are running out of memory, consider de

In [21]:
tokenizer = transformers.AutoTokenizer.from_pretrained(model_id)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [16]:
sentence = debug_sentences[1]["sentence"]
print(sentence)
people = get_people(sentence)
prompt = tokenizer.apply_chat_template([{'role': 'user', 'content': format_people_prompt(sentence=sentence, name='Kay Ryan', address='256 Overflow St', specialty='neurology')}], tokenize=False)
print(f'Prompt ({len(tokenizer(prompt)["input_ids"])} tokens): ```{prompt}```')

sampling_params = vllm.SamplingParams(temperature=1.0, max_tokens=1024, n=25)
response = model.generate(prompt, sampling_params=sampling_params)[0]
for i, output in enumerate(response.outputs, start=1):
    print(f'Generated Query {i}: ```{output.text}```')

Dr. Kay Ryan's office at 321 Pine St. in Baltimore is so welcoming and efficient. Highly recommend! #CityCare
Prompt (517 tokens): ```<|begin_of_text|><|start_header_id|>user<|end_header_id|>

Write Julia code to query a PClean table of records about doctors based on the given input sentence.

In general, your output should look like pairs of lines:

```julia
blah_key = PClean.resolve_dot_expression(trace.model, :Obs, :blah_col)
row_trace[blah_key] = "Value of Blah as expressed in the sentence"
```

The dataset has just four columns to query:

- :name (the doctor's full name, first and last)
- :city (the city where the doctor practices)
- :address (the doctor's office address)
- :specialty (the doctor's specialty)

Please generate code to query all values specified in the sentence. Output the Julia code directly with no preamble or commentary. Write just two lines per column.

Input: Loved visiting Dr. Kay Ryan's neurology office at 256 Overflow St! No wait time at all. #Baltimore
Outp

Processed prompts: 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:12<00:00, 12.99s/it, est. speed input: 39.79 toks/s, output: 255.07 toks/s]

Generated Query 1: ```<|start_header_id|>assistant<|end_header_id|>

Here is the Julia code to query the PClean table of records about doctors based on the given input sentence:

```julia
name_key = PClean.resolve_dot_expression(trace.model, :Obs, :name)
row_trace[name_key] = "Kay Ryan"
address_key = PClean.resolve_dot_expression(trace.model, :Obs, :address)
row_trace[address_key] = "321 Pine St"
city_key = PClean.resolve_dot_expression(trace.model, :Obs, :city)
row_trace[city_key] = "Baltimore"
``````
Generated Query 2: ```<|start_header_id|>assistant<|end_header_id|>

Here is the Julia code to query the PClean table:

```julia
name_key = PClean.resolve_dot_expression(trace.model, :Obs, :name)
row_trace[name_key] = "Kay Ryan"
address_key = PClean.resolve_dot_expression(trace.model, :Obs, :address)
row_trace[address_key] = "321 Pine St"
city_key = PClean.resolve_dot_expression(trace.model, :Obs, :city)
row_trace[city_key] = "Baltimore"
``````
Generated Query 3: ```<|start_header_id|>as

In [43]:
for sentence_datum in detailed_debug_sentences:
    sentence = sentence_datum["sentence"]
    print(sentence)
    prompt = tokenizer.apply_chat_template([{'role': 'user', 'content': format_json_prompt(sentence=sentence)}], tokenize=False)
    print(f'Prompt ({len(tokenizer(prompt)["input_ids"])} tokens): ```{prompt}```')
    
    sampling_params = vllm.SamplingParams(temperature=1.0, max_tokens=1024, n=25)
    response = model.generate(prompt, sampling_params=sampling_params)[0]
    for i, output in enumerate(response.outputs, start=1):
        print(f'Generated Query {i}: ```{output.text}```')

Just had an insightful consultation with Tucker Skincare Inc.'s dermatologist Kay Ryan at 635 Golden Drive in Little Rock (LI-722). Feeling optimistic about my health! #HealthMatters
Prompt (557 tokens): ```<|begin_of_text|><|start_header_id|>user<|end_header_id|>

Write a flat JSON object describing the doctor in the given input sentence.

In general, your output should look like:

```json
{
    "some_feature": "foo",
    "another_feature": "bar",
    ...
    "last_feature": "baz"
}
```

There are seven features we would like to extract from the sentence:

- "first_name" (the doctor's first name)
- "last_name" (the doctor's last name)
- "c2z3" (a code of the form XX-YYY where XX are letters and YYY are digits)
- "address" (the doctor's office address, "line 1" -- this is the street address)
- "address2" (additional address information giving a suite number, unit number, et cetera -- "line 2" of the address)
- "specialty" (the doctor's specialty)
- "legal_name" (the legal name of the d

Processed prompts: 100%|███| 1/1 [00:03<00:00,  3.78s/it, est. speed input: 147.56 toks/s, output: 455.91 toks/s]


Generated Query 1: ```<|start_header_id|>assistant<|end_header_id|>

{
  "first_name": "Kay", 
  "last_name": "Ryan", 
  "specialty": "dermatologist", 
  "legal_name": "Tucker Skincare Inc.", 
  "address": "635 Golden Drive", 
  "c2z3": "LI-722"
}```
Generated Query 2: ```<|start_header_id|>assistant<|end_header_id|>

{
  "first_name": "Kay", 
  "last_name": "Ryan", 
  "specialty": "dermatologist", 
  "legal_name": "Tucker Skincare Inc.", 
  "address": "635 Golden Drive", 
  "c2z3": "LI-722"
}```
Generated Query 3: ```<|start_header_id|>assistant<|end_header_id|>

{
  "first_name": "Kay", 
  "last_name": "Ryan", 
  "specialty": "dermatologist", 
  "legal_name": "Tucker Skincare Inc.", 
  "address": "635 Golden Drive", 
  "c2z3": "LI-722"
}```
Generated Query 4: ```<|start_header_id|>assistant<|end_header_id|>

{
  "first_name": "Kay", 
  "last_name": "Ryan", 
  "specialty": "dermatologist", 
  "legal_name": "Tucker Skincare Inc.", 
  "address": "635 Golden Drive", 
  "c2z3": "LI-722"
}

Processed prompts: 100%|███| 1/1 [00:03<00:00,  3.91s/it, est. speed input: 143.64 toks/s, output: 440.91 toks/s]


Generated Query 1: ```<|start_header_id|>assistant<|end_header_id|>

{
  "first_name": "Kay", 
  "last_name": "Ryan", 
  "specialty": "gastroenterology", 
  "address": "321 Pine St", 
  "c2z3": "ME-852", 
  "legal_name": "Get Guts Inc"
}```
Generated Query 2: ```<|start_header_id|>assistant<|end_header_id|>

{
  "first_name": "Kay", 
  "last_name": "Ryan", 
  "specialty": "gastroenterology", 
  "address": "321 Pine St", 
  "c2z3": "ME-852", 
  "legal_name": "Get Guts Inc"
}```
Generated Query 3: ```<|start_header_id|>assistant<|end_header_id|>

{
  "first_name": "Kay", 
  "last_name": "Ryan", 
  "specialty": "gastroenterology", 
  "address": "321 Pine St", 
  "c2z3": "ME-852", 
  "legal_name": "Get Guts Inc"
}```
Generated Query 4: ```<|start_header_id|>assistant<|end_header_id|>

{
  "first_name": "Kay", 
  "last_name": "Ryan", 
  "specialty": "gastroenterology", 
  "address": "321 Pine St", 
  "c2z3": "ME-852", 
  "legal_name": "Get Guts Inc"
}```
Generated Query 5: ```<|start_header

...

Generated Query 1: ```<|start_header_id|>assistant<|end_header_id|>

{
  "first_name": "Kay", 
  "last_name": "Ryan", 
  "specialty": "psychiatrist", 
  "address": "407 Windings Way, Building 4", 
  "c2z3": "NO-232", 
  "legal_name": "Good Babylonian Services Inc"```
Generated Query 2: ```<|start_header_id|>assistant<|end_header_id|>

{
  "first_name": "Kay", 
  "last_name": "Ryan", 
  "specialty": "psychiatrist", 
  "address": "407 Windings Way, Building 4", 
  "c2z3": "NO-232", 
  "legal_name": "Good Babylonian Services Inc"```
Generated Query 3: ```<|start_header_id|>assistant<|end_header_id|>

{
  "first_name": "Kay", 
  "last_name": "Ryan", 
  "specialty": "psychiatrist", 
  "address": "407 Windings Way, Building 4", 
  "c2z3": "NO-232", 
  "legal_name": "Good Babylonian Services Inc"```
Generated Query 4: ```<|start_header_id|>assistant<|end_header_id|>

{
  "first_name": "Kay", 
  "last_name": "Ryan", 
  "specialty": "psychiatrist", 
  "address": "407 Windings Way, Building 4", 

Processed prompts: 100%|███| 1/1 [00:04<00:00,  4.10s/it, est. speed input: 140.09 toks/s, output: 445.18 toks/s]


Generated Query 1: ```<|start_header_id|>assistant<|end_header_id|>

{
  "first_name": "Kay", 
  "last_name": "Ryan", 
  "specialty": "endocrinology", 
  "c2z3": "PA-922", 
  "address": "634 Bronco Lane, Building 2", 
  "legal_name": "Healthy Mojave Inc."```
Generated Query 2: ```<|start_header_id|>assistant<|end_header_id|>

{
  "first_name": "Kay", 
  "last_name": "Ryan", 
  "specialty": "endocrinology", 
  "c2z3": "PA-922", 
  "legal_name": "Healthy Mojave Inc.", 
  "address": "634 Bronco Lane, Building 2"```
Generated Query 3: ```<|start_header_id|>assistant<|end_header_id|>

{
  "first_name": "Kay", 
  "last_name": "Ryan", 
  "specialty": "endocrinology", 
  "c2z3": "PA-922", 
  "legal_name": "Healthy Mojave Inc.", 
  "address": "634 Bronco Lane, Building 2"
}```
Generated Query 4: ```<|start_header_id|>assistant<|end_header_id|>

{
  "first_name": "Kay", 
  "last_name": "Ryan", 
  "specialty": "endocrinology", 
  "c2z3": "PA-922", 
  "legal_name": "Healthy Mojave Inc.", 
  "addre

Processed prompts: 100%|███| 1/1 [00:04<00:00,  4.08s/it, est. speed input: 139.87 toks/s, output: 429.91 toks/s]

Generated Query 1: ```<|start_header_id|>assistant<|end_header_id|>

{
  "first_name": "Kay", 
  "last_name": "Ryan", 
  "specialty": "pediatrician", 
  "legal_name": "Vinnie and Brewster Health Services Inc", 
  "address": "303 Park Place", 
  "c2z3": "SO-490"
}```
Generated Query 2: ```<|start_header_id|>assistant<|end_header_id|>

{
  "first_name": "Kay", 
  "last_name": "Ryan", 
  "specialty": "pediatrician", 
  "legal_name": "Vinnie and Brewster Health Services Inc", 
  "address": "303 Park Place", 
  "c2z3": "SO-490"
}```
Generated Query 3: ```<|start_header_id|>assistant<|end_header_id|>

{
  "first_name": "Kay", 
  "last_name": "Ryan", 
  "specialty": "pediatrician", 
  "legal_name": "Vinnie and Brewster Health Services Inc", 
  "address": "303 Park Place", 
  "c2z3": "SO-490"
}```
Generated Query 4: ```<|start_header_id|>assistant<|end_header_id|>

{
  "first_name": "Kay", 
  "last_name": "Ryan", 
  "specialty": "pediatrician", 
  "legal_name": "Vinnie and Brewster Health Serv

In [ ]:
try:
    del model
    gc.collect()
except NameError:
    pass

### Running the model with constraints

In [17]:
sentence = debug_sentences[1]["sentence"]
prompt = tokenizer.apply_chat_template(
    [
        {'role': 'user', 'content': format_people_prompt(sentence=sentence, name='Kay Ryan', address='256 Overflow St', specialty='neurology')}
    ],
    tokenize=False,
)
print(f'Prompt ({len(tokenizer(prompt)["input_ids"])} tokens): ```{prompt}```')


# ignored top_p=0.95
server_inference_params = {'max_tokens': 128, 'n_particles': 15, 'temperature': 1.0, 'grammar': pclean_grammar}
response = run_inference_server(prompt, **server_inference_params)
try:
    data = response.json()
    for i, (query, likelihood) in enumerate(data['posterior'].items(), start=1):
        
        print(f'Generated Query {i} (likelihood {100 * likelihood:.2f}%): ```{query}```')
except json.JSONDecodeError:
    print(response.status_code)
    print(response.text)

Prompt (517 tokens): ```<|begin_of_text|><|start_header_id|>user<|end_header_id|>

Write Julia code to query a PClean table of records about doctors based on the given input sentence.

In general, your output should look like pairs of lines:

```julia
blah_key = PClean.resolve_dot_expression(trace.model, :Obs, :blah_col)
row_trace[blah_key] = "Value of Blah as expressed in the sentence"
```

The dataset has just four columns to query:

- :name (the doctor's full name, first and last)
- :city (the city where the doctor practices)
- :address (the doctor's office address)
- :specialty (the doctor's specialty)

Please generate code to query all values specified in the sentence. Output the Julia code directly with no preamble or commentary. Write just two lines per column.

Input: Loved visiting Dr. Kay Ryan's neurology office at 256 Overflow St! No wait time at all. #Baltimore
Output: ```julia
name_key = PClean.resolve_dot_expression(trace.model, :Obs, :name)
row_trace[name_key] = "Kay Rya

In [41]:
sentence = debug_sentences[1]["sentence"]
prompt = tokenizer.apply_chat_template(
    [
        {'role': 'user', 'content': format_json_prompt(sentence=sentence)}
    ],
    tokenize=False,
)
print(f'Prompt ({len(tokenizer(prompt)["input_ids"])} tokens): ```{prompt}```')


# ignored top_p=0.95
server_inference_params = {'max_tokens': 128, 'n_particles': 15, 'temperature': 1.0, 'grammar': json_grammar}
response = run_inference_server(prompt, **server_inference_params)
try:
    data = response.json()
    for i, (query, likelihood) in enumerate(data['posterior'].items(), start=1):
        print(f'Generated Query {i} (likelihood {100 * likelihood:.2f}%): ```{query}```')
except json.JSONDecodeError:
    print(response.status_code)
    print(response.text)

Prompt (544 tokens): ```<|begin_of_text|><|start_header_id|>user<|end_header_id|>

Write a flat JSON object describing the doctor in the given input sentence.

In general, your output should look like:

```json
{
    "some_feature": "foo",
    "another_feature": "bar",
    ...
    "last_feature": "baz"
}
```

There are seven features we would like to extract from the sentence:

- "first_name" (the doctor's first name)
- "last_name" (the doctor's last name)
- "c2z3" (a code of the form XX-YYY where XX are letters and YYY are digits)
- "address" (the doctor's office address, "line 1" -- this is the street address)
- "address2" (additional address information giving a suite number, unit number, et cetera -- "line 2" of the address)
- "specialty" (the doctor's specialty)
- "legal_name" (the legal name of the doctor's business, practice, or employer)

Please generate a key-value pair for every such value specified in the sentence. Avoid nulls and empty strings. Omit any missing values. Do n

In [23]:
responses = {}
for sentence_datum in debug_sentences:
    sentence = sentence_datum["sentence"]
    prompt = tokenizer.apply_chat_template(
        [
            {'role': 'user', 'content': format_people_prompt(sentence=sentence, name='Kay Ryan', address='256 Overflow St', specialty='neurology')}
        ],
        tokenize=False,
    )
    logger.debug('Prompt (%d tokens): ```%s```', len(tokenizer(prompt)["input_ids"]), prompt)

    # ignored top_p=0.95
    server_inference_params = {'max_tokens': 256, 'n_particles': 15, 'temperature': 1.0, 'grammar': pclean_grammar}
    response = run_inference_server(prompt, **server_inference_params)
    responses[sentence] = response

In [44]:
def md_html_output_text(text: str) -> str:
    return text.replace("<", "&lt;").replace(">", "&gt;").replace("|", "\|").replace("\n", "<br>").replace("```", "\\```")

table_rows = ['| Sentence | Extracted Code | Raw Response | Aggregate Likelihood (Raw LH) |', '|:--- |:--- |:--- | ---:|']
for sentence, response in responses.items():
    try:
        data = response.json()
        posterior = data['posterior']
        aggregate_likelihoods = get_aggregate_likelihoods(posterior)
        # for i, (output_text, likelihood) in enumerate(posterior.items(), start=1):
        #     # Assert the response parses properly
        #     pclean_parser.parse(output_text)

        #     code = extract_code_from_response(output_text)
        #     aggregate_likelihood = aggregate_likelihoods[code]
        #     code_with_br = code.replace("\n", "<br>")
        #     output_with_br = md_html_output_text(output_text)
        #     table_rows.append(
        #         f'| {sentence if i == 1 else "   "} '
        #         f'| <pre>{code_with_br}</pre> '
        #         f'| <pre>{output_with_br}</pre> '
        #         f'| {100 * aggregate_likelihood:.2f}% ({100 * likelihood:.2f}%) |'
        #     )
        for i, (code, likelihood) in enumerate(aggregate_likelihoods.items(), start=1):
            aggregate_likelihood = aggregate_likelihoods[code]
            code_with_br = code.replace("\n", "<br>")
            table_rows.append(
                f'| {sentence if i == 1 else "   "} '
                f'| <pre>{code_with_br}</pre> '
                f'| (unk) '
                f'| {100 * aggregate_likelihood:.2f}% ({100 * likelihood:.2f}%) |'
            )
    except json.JSONDecodeError:
        table_rows.append(
            f'| {sentence if i == 1 else "   "} | ERROR: {responses.status_code} - {response.text} | 100% |'
        )
display(Markdown('\n'.join(table_rows)))

NameError: name 'responses' is not defined

In [27]:
responses = {}
for sentence_datum in tqdm([#{"sentence": "John Smith the podiatrist is terrible, so much worse than John Smith the cardiologist."}, {"sentence": "John Smith works with Jack Smith at the neurology office. He’s terrible at neurology."},
                            {"sentence": "John Smith's office (Happy Brain Services LLC) at 512 Example Street Suite 3600 (CA-170) is terrible!"}]):
    sentence = sentence_datum["sentence"]
    prompt = tokenizer.apply_chat_template(
        [
            {'role': 'user', 'content': format_json_prompt(sentence=sentence)}
        ],
        tokenize=False,
    )
    logger.debug('Prompt (%d tokens): ```%s```', len(tokenizer(prompt)["input_ids"]), prompt)

    # ignored top_p=0.95
    server_inference_params = {'max_tokens': 256, 'n_particles': 15, 'temperature': 1.0, 'grammar': json_grammar}
    response = run_inference_server(prompt, **server_inference_params)
    responses[sentence] = response
    #break

100%|██████████████████████████████████████████████████████████████████████████████| 1/1 [00:29<00:00, 29.94s/it]


In [28]:
def md_html_output_text_for_json(text: str) -> str:
    return text.replace("<", "&lt;").replace(">", "&gt;").replace("|", "\|").replace("\n", "<br>").replace("```", "\\```")

table_rows = ['| Sentence | Extracted JSON | Raw Response | Aggregate Likelihood (Raw LH) |', '|:--- |:--- |:--- | ---:|']
for sentence, response in responses.items():
    try:
        data = response.json()
        posterior = data['posterior']
        for i, (output_text, likelihood) in enumerate(posterior.items(), start=1):
            # Assert the response parses properly
            json_parser.parse(output_text)

            code = extract_code_from_response(output_text)
            code_with_br = code.replace("\n", "<br>")
            output_with_br = md_html_output_text_for_json(output_text)
            table_rows.append(
                f'| {sentence if i == 1 else "   "} '
                f'| <pre>{code_with_br}</pre> '
                f'| <pre>{output_with_br}</pre> '
                f'| {100 * likelihood:.2f}% ({100 * likelihood:.2f}%) |'
            )
    except json.JSONDecodeError:
        table_rows.append(
            f'| {sentence if i == 1 else "   "} | ERROR: {responses.status_code} - {response.text} | 100% |'
        )
display(Markdown('\n'.join(table_rows)))

| Sentence | Extracted JSON | Raw Response | Aggregate Likelihood (Raw LH) |
|:--- |:--- |:--- | ---:|
| John Smith's office (Happy Brain Services LLC) at 512 Example Street Suite 3600 (CA-170) is terrible! | <pre>{<br>  "first_name": "John",<br>  "last_name": "Smith",<br>  "c2z3": "CA-170",<br>  "address": "512 Example Street",<br>  "address2": "Suite 3600",<br>  "legal_name": "Happy Brain Services LLC",<br>  "specialty": "general"<br>}</pre> | <pre>&lt;\|start_header_id\|&gt;assistant&lt;\|end_header_id\|&gt;<br><br>\```<br>{<br>  "first_name": "John",<br>  "last_name": "Smith",<br>  "c2z3": "CA-170",<br>  "address": "512 Example Street",<br>  "address2": "Suite 3600",<br>  "legal_name": "Happy Brain Services LLC",<br>  "specialty": "general"<br>}<br>\```</pre> | 83.33% (83.33%) |
|     | <pre>{<br>  "first_name": "John",<br>  "last_name": "Smith",<br>  "c2z3": "CA-170",<br>  "address": "512 Example Street",<br>  "address2": "Suite 3600",<br>  "legal_name": "Happy Brain Services LLC",<br>  "specialty": "office"<br>}</pre> | <pre>&lt;\|start_header_id\|&gt;assistant&lt;\|end_header_id\|&gt;<br><br>\```<br>{<br>  "first_name": "John",<br>  "last_name": "Smith",<br>  "c2z3": "CA-170",<br>  "address": "512 Example Street",<br>  "address2": "Suite 3600",<br>  "legal_name": "Happy Brain Services LLC",<br>  "specialty": "office"<br>}<br>\```</pre> | 16.67% (16.67%) |
|     | <pre>{<br>    "first_name": "John",<br>    "last_name": "Smith",<br>    "c2z3": "CA-170",<br>    "address": "512 Example Street",<br>    "address2": "Suite 3600",<br>    "legal_name": "Happy Brain Services LLC",<br>    "specialty": "no specialty"<br>}</pre> | <pre>&lt;\|start_header_id\|&gt;assistant&lt;\|end_header_id\|&gt;<br><br>\```json<br>{<br>    "first_name": "John",<br>    "last_name": "Smith",<br>    "c2z3": "CA-170",<br>    "address": "512 Example Street",<br>    "address2": "Suite 3600",<br>    "legal_name": "Happy Brain Services LLC",<br>    "specialty": "no specialty"<br>}<br>\```</pre> | 0.00% (0.00%) |